# 職缺資料清理與寫入 Supabase

依 `cleaner步驟_v2.md` 清理 `jobs_rows.csv`，並依 ERD（`career_pilot_ERD_欄位對齊總表.md`）寫入 `company_info`、`job_posting`。

**執行順序**：請由上面依序跑到下面，可逐步執行各階段並測試。  
**工作目錄**：Notebook 請在 `supabase_control` 下開啟，確保 `jobs_rows.csv`、`Erd/.env` 路徑正確。

## 前置準備：安裝套件、載入資料

- 套件：`pandas`, `numpy`, `supabase`, `python-dotenv`（見 `pyproject.toml`）
- 資料：`jobs_rows.csv`

In [2]:
import pandas as pd
import numpy as np
import re
import json
from pathlib import Path

DATA_DIR = Path(".")
RAW_CSV = DATA_DIR / "jobs_rows.csv"

df_raw = pd.read_csv(RAW_CSV)
print(f"✓ 載入 {len(df_raw):,} 筆原始資料")
print(f"欄位: {list(df_raw.columns)}")
df_raw.head(2)

✓ 載入 2,663 筆原始資料
欄位: ['job_id', 'job_name', 'company_name', 'company_url', 'update_date', 'url', 'status', 'job_description', 'job_category', 'salary', 'job_type', 'location', 'management', 'business_trip', 'work_time', 'vacation', 'start_work', 'headcount', 'work_exp', 'education', 'major', 'language', 'skills', 'tools', 'certificates', 'other_requirements', 'legal_benefits', 'other_benefits', 'raw_benefits', 'contact_info', 'actively_hiring', 'applicants', 'created_at']


,job_id,job_name,company_name,company_url,update_date,url,status,job_description,job_category,salary,...,tools,certificates,other_requirements,legal_benefits,other_benefits,raw_benefits,contact_info,actively_hiring,applicants,created_at
0,109g12026-01-21,C#軟體工程師,泛太資訊科技開發股份有限公司,NaN,2026-01-21,https://www.104.com.tw/job/109g1?jobsource=job...,active,- 熟悉C#、.NET 、ASP.Net\n-依據系統需求規格進行程式設計與開發\n-使用 ...,軟體工程師、Internet程式設計師、系統分析師,待遇面議,...,Windows 2003、Windows XP、ASP.NET、C#、Visual Basi...,NaN,NaN,週休二日、勞保、健保、職災保險,年終獎金、三節獎金/禮品、專業證照獎金、結婚禮金、生育津貼、員工進修補助、國內旅遊、部門聚餐...,泛太資訊遵循勞基法之相關規定並提供下列福利\n\n►黃金地段辦公室！國父紀念館站１號出口，輕...,陳小姐\n02-27313250-202\n本職務設定3個工作天回覆,False,0~5,2026-01-28 09:02:28.359263+00
1,1x89f2026-01-26,研發替代役-軟體工程師,凌群電腦股份有限公司,NaN,2026-01-26,https://www.104.com.tw/job/1x89f?jobsource=job...,active,軟體開發與設計、架構設計、資料設計、系統整合、軟體產品研發、技術管理、專案管理\n\n※薪資...,軟體工程師、系統分析師、Internet程式設計師,"月薪37,000元以上取得專屬你的薪水報告",...,NaN,NaN,1.研究所以上資訊相關系所畢\n2.熟稔Java或.Net語言\n3.具系統分析設計與程式撰...,哺乳室、週休二日、家庭照顧假、勞保、健保、陪產假、產假、特別休假、育嬰留停、女性生理假、勞退...,生育津貼、社團補助、員工進修補助、部門聚餐、社團活動、特約商店、內部講師鐘點費、員工團體保險,人才是凌群最重要的資產 \n用心照護。安心工作\n\n【凌群福利措施】\n\n★完善的保險保...,李小姐\n誠摯邀請您至公司網站www.syscom.com.tw 更進一步認識凌群，並期待與...,False,0~5,2026-01-28 09:26:33.981361+00


## 階段一：資料探索與品質檢查

- 缺失值統計、重複檢查、job_id 格式、公司名稱統計、薪資檢查
- **headcount**：職缺需求人數，非公司規模，清理時不解析

In [3]:
# 1. 缺失值統計
missing = df_raw.isna().sum()
missing_pct = (missing / len(df_raw) * 100).round(1)
miss_df = pd.DataFrame({"缺失數": missing, "缺失%": missing_pct})
print("【缺失值統計】")
display(miss_df[miss_df["缺失數"] > 0].sort_values("缺失數", ascending=False))

# 2. 重複檢查
dup_all = df_raw.duplicated().sum()
dup_cnj = df_raw.duplicated(subset=["company_name", "job_name"], keep="first").sum()
has_url = df_raw["url"].notna() & (df_raw["url"].astype(str).str.len() > 0)
dup_url = df_raw.loc[has_url].duplicated(subset=["url", "job_name"], keep="first").sum()
print(f"\n【重複】 完全重複: {dup_all:,} | (company_name, job_name): {dup_cnj:,} | (url, job_name) 有 url 時: {dup_url:,}")

# 3. job_id 格式（異常範例）
jid = df_raw["job_id"].astype(str)
weird = jid[~jid.str.match(r"^[a-zA-Z0-9\-]+$", na=False)]
print(f"\n【job_id 異常】 {len(weird):,} 筆")
if len(weird) > 0:
    display(weird.head())

# 4. 公司統計
n_company = df_raw["company_name"].nunique()
top = df_raw["company_name"].value_counts().head(5)
print(f"\n【公司】 不重複 {n_company:,} 家；職缺數前 5:")
display(top)

# 5. 薪資範例
print("\n【薪資欄 salary 範例】")
display(df_raw["salary"].dropna().head(10))

【缺失值統計】


,缺失數,缺失%
company_url,2596,97.5
certificates,2574,96.7
skills,1221,45.9
other_benefits,926,34.8
other_requirements,908,34.1
legal_benefits,863,32.4
tools,570,21.4
raw_benefits,139,5.2
business_trip,83,3.1
management,83,3.1



【重複】 完全重複: 0 | (company_name, job_name): 5 | (url, job_name) 有 url 時: 0

【job_id 異常】 0 筆

【公司】 不重複 1,611 家；職缺數前 5:


company_name
鴻海精密工業股份有限公司                            29
和碩集團_和碩聯合科技股份有限公司                       22
工研院 _財團法人工業技術研究院                        19
台達電子工業股份有限公司 _DELTA ELECTRONICS INC.    19
GARMIN_台灣國際航電股份有限公司                     18
Name: count, dtype: int64


【薪資欄 salary 範例】


0                          待遇面議
1         月薪37,000元以上取得專屬你的薪水報告
2    月薪35,000~50,000元取得專屬你的薪水報告
3    月薪35,000~75,000元取得專屬你的薪水報告
4                          待遇面議
5                          待遇面議
6         月薪36,000元以上取得專屬你的薪水報告
7                          待遇面議
8                          待遇面議
9    月薪32,000~54,000元取得專屬你的薪水報告
Name: salary, dtype: str

In [ ]:
# 產業分類測試（industry_classification_optimization.md 測試用例）
assert extract_industry("日月光半導體製造股份有限公司") == "製造業"
assert extract_industry("台積電科技股份有限公司") == "製造業"
assert extract_industry("中山醫學大學附設醫院") == "醫療"
assert extract_industry("沐恩生醫光電股份有限公司") == "製造業"
assert extract_industry("國泰金融控股股份有限公司") == "金融業"
assert extract_industry("趨勢科技股份有限公司") == "資訊科技"
assert extract_industry("聯發科技股份有限公司") == "資訊科技"
print("✓ 所有測試用例通過")

## 階段二：資料清理函數定義

依步驟定義：`clean_text`、`extract_industry`(公司名稱)、`infer_industry_from_job_category`(職缺類別 fallback)、`standardize_location`→(full_address, city, district)、`clean_salary`、`determine_remote_option`、`merge_requirements`、`create_job_details`。

In [14]:
def clean_text(text):
    """清理文字：移除多餘空白、換行、特殊字元；處理 None/NaN。"""
    if pd.isna(text) or text is None:
        return None
    s = str(text).strip()
    s = re.sub(r"[\r\n]+", " ", s)
    s = re.sub(r"\s+", " ", s)
    return s if s else None


# 組織後綴：移除後再進行關鍵字匹配
ORG_SUFFIXES = ["股份有限公司", "有限公司", "集團", "分公司", "財團法人"]

def clean_company_name(company_name):
    """移除公司名稱中的組織後綴（股份有限公司、有限公司、集團、分公司等）。"""
    if pd.isna(company_name) or not str(company_name).strip():
        return None
    name = str(company_name).strip()
    for suffix in ORG_SUFFIXES:
        if name.endswith(suffix):
            name = name[: -len(suffix)].strip()
        while suffix in name:
            name = name.replace(suffix, " ").strip()
    return name if name else None


# 分層關鍵字優先級：特定領域 > 複合領域 > 通用關鍵字
PRIORITY_LAYERS = {
    "tier_1_specific": [
        ("製造業", ["半導體製造", "製程", "產線", "封裝測試", "晶圓", "fab", "光電", "台積電"]),
        ("醫療", ["醫療器材", "醫藥", "生技", "藥廠", "醫檢", "診所", "醫院", "醫材", "醫電", "生醫", "製藥", "藥業"]),
        ("金融業", ["銀行", "保險", "證券", "投信", "金控", "金融", "控股", "資產管理"]),
    ],
    "tier_2_mixed": [
        ("資訊科技", ["軟體", "網路服務", "雲端", "ai應用", "資安", "系統整合"]),
        ("製造業", ["材料", "精密", "機械"]),
    ],
    "tier_3_generic": [
        ("資訊科技", ["科技", "資訊", "數位"]),
        ("服務業", ["服務"]),
    ],
}


def extract_industry(company_name, job_category=None):
    """從公司名稱推斷產業類別（分層關鍵字優先級，向後相容保留原名）。"""
    if pd.isna(company_name) or not str(company_name).strip():
        return None
    name = clean_company_name(company_name)
    if not name:
        return None
    name_lower = name.lower()
    # Tier 1：匹配即直接返回（人力銀行為求職平台，不以「銀行」判為金融業）
    for industry, keywords in PRIORITY_LAYERS["tier_1_specific"]:
        kws = [kw for kw in keywords if not (industry == "金融業" and kw == "銀行" and "人力銀行" in name_lower)]
        if any(kw in name_lower for kw in kws):
            return industry
    # Tier 2、3：記錄候選，優先取 Tier2
    for industry, keywords in PRIORITY_LAYERS["tier_2_mixed"]:
        if any(kw in name_lower for kw in keywords):
            return industry
    for industry, keywords in PRIORITY_LAYERS["tier_3_generic"]:
        if any(kw in name_lower for kw in keywords):
            return industry
    if job_category is not None and str(job_category).strip():
        return infer_industry_from_job_category(job_category)
    return None


def infer_industry_from_job_category(job_categories_text):
    """從職缺類別（job_category 彙總字串）推斷產業；公司名稱無匹配時 fallback 用。"""
    if pd.isna(job_categories_text) or not str(job_categories_text).strip():
        return None
    s = str(job_categories_text).strip().lower()
    manufacturing_kw = [
        "製造", "產線", "設備", "製程", "機構工程", "半導體", "光電", "pcb", "smt",
        "品保", "倉管", "生產", "焊接", "cnc", "製程工程師", "設備工程師", "生產管理",
        "品管", "qc", "qe", "ie", "me", "廠務", "生產線", "作業員",
    ]
    medical_kw = [
        "醫護", "護理", "藥師", "醫檢", "醫事", "醫師", "護理師", "醫檢師",
        "醫療器材", "醫學工程", "臨床", "復健",
    ]
    keywords = [
        ("資訊科技", ["軟體", "程式", "系統分析", "internet", "mis", "韌體", "資料庫", "資安", "演算法", "dba", "bios", "全端", "後端", "前端", "資料科學", "大數據", "雲端", "devops", "sre", "嵌入式", "網管", "qa"]),
        ("製造業", manufacturing_kw),
        ("金融業", ["金融", "銀行", "保險", "證券", "理財", "風控", "精算", "授信", "櫃員"]),
        ("醫療", medical_kw),
        ("行銷", ["行銷", "廣告", "媒體", "電商", "社群", "文案", "企劃", "數位行銷"]),
        ("商業", ["人資", "人力資源", "會計", "財務", "審計", "業務", "客服", "採購", "行政", "總務", "秘書", "法務", "顧問"]),
        ("設計", ["設計", "ui", "ux", "平面", "工業設計", "視覺"]),
        ("教育", ["教師", "講師", "教練", "補習", "教材", "教學"]),
    ]
    for industry, kws in keywords:
        if any(kw in s for kw in kws):
            return industry
    return None


import re

CITIES = [
    "台北市", "新北市", "桃園市", "台中市", "台南市", "高雄市", 
    "基隆市", "新竹市", "嘉義市",
    "新竹縣", "苗栗縣", "彰化縣", "南投縣", "雲林縣", "嘉義縣", 
    "屏東縣", "宜蘭縣", "花蓮縣", "台東縣", "澎湖縣", "金門縣", "連江縣",
]

# 定義區/鄉/鎮/市的正則表達式
DISTRICT_SUFFIX = re.compile(r"^(.+?[區鄉鎮市])")

VALID_DISTRICTS = {"東區", "北區", "香山區", "西區"}
GARDEN_BLACKLIST = ["工業園區", "科學園區", "園區", "太空中心"]  # 加入太空中心

def standardize_location(city, district, location):
    """輸出 (full_address, city, district)。"""
    parts = [x for x in [city, district, location] if pd.notna(x) and str(x).strip()]
    full = "".join(str(p).strip() for p in parts) if parts else None
    if not full:
        return None, None, None
    
    city_val, district_val = None, None
    
    for c in CITIES:
        if full.startswith(c):
            city_val = c
            rest = full[len(c):].strip()
            
            # 處理重複縣市名（如「新竹市新竹市...」）
            if rest.startswith(c):
                rest = rest[len(c):].strip()
            
            # 新竹市/嘉義市特殊處理
            if city_val in ["新竹市", "嘉義市"]:
                for valid_d in VALID_DISTRICTS:
                    if rest.startswith(valid_d):
                        district_val = valid_d
                        break
            
            # 一般縣市處理
            else:
                m = DISTRICT_SUFFIX.match(rest)
                if m:
                    candidate = m.group(1).strip()
                    if not any(b in candidate for b in GARDEN_BLACKLIST):
                        district_val = candidate
                
                if not district_val and rest:
                    tok = re.match(r"^([^\d路街段巷弄號]+?[區鄉鎮市])", rest)
                    if tok:
                        candidate = tok.group(1).strip()
                        if not any(b in candidate for b in GARDEN_BLACKLIST):
                            district_val = candidate
            break
    
    return full, city_val, district_val

def _parse_int(s):
    if not s:
        return None
    return int(re.sub(r"[,，\s]", "", str(s)))

def clean_salary(salary_raw):
    """薪資清理 → (min, max)。待遇面議→(40000,40000)；區間解析 min,max。"""
    if pd.isna(salary_raw) or not str(salary_raw).strip():
        return None, None
    s = str(salary_raw).strip()
    if "面議" in s or "面谈" in s:
        return 40000, 40000
    s_clean = re.sub(r"[,，\s]", "", s)
    mm = re.search(r"([\d,]+)\s*[~～\-－至到]\s*([\d,]+)", s)
    if mm:
        lo, hi = _parse_int(mm.group(1)), _parse_int(mm.group(2))
        if lo is not None and hi is not None:
            return (min(lo, hi), max(lo, hi))
    single = re.search(r"月薪\s*([\d,]+)", s)
    if single:
        v = _parse_int(single.group(1))
        if v is not None:
            return (v, v)
    num = re.search(r"(\d+)\s*元", s_clean)
    if num:
        v = int(num.group(1))
        return (v, 999999)
    return None, None


def determine_remote_option(addr, job_type):
    """判斷 remote / hybrid / onsite。可傳 full_address 與 job_type。"""
    if pd.notna(job_type) and "遠端" in str(job_type):
        return "remote"
    if pd.notna(job_type) and " hybrid" in str(job_type).lower():
        return "hybrid"
    addr_str = str(addr) if pd.notna(addr) else ""
    if "遠端" in addr_str or "remote" in addr_str.lower():
        return "remote"
    if "混合" in addr_str or "hybrid" in addr_str.lower():
        return "hybrid"
    return "onsite"


def merge_requirements(row):
    """合併 work_exp, education, major, language, skills, tools, certificates, other_requirements。"""
    keys = ["work_exp", "education", "major", "language", "skills", "tools", "certificates", "other_requirements"]
    parts = []
    for k in keys:
        v = row.get(k)
        if pd.notna(v) and str(v).strip():
            parts.append(str(v).strip())
    return "\n".join(parts) if parts else None


def create_job_details(row):
    """建立 job_details JSON。"""
    keys = ["work_time", "vacation", "start_work", "business_trip", "legal_benefits", "other_benefits", "raw_benefits"]
    d = {}
    for k in keys:
        v = row.get(k)
        if pd.notna(v) and str(v).strip():
            d[k] = str(v).strip()
    return d if d else None

print("✓ 清理函數已定義")

✓ 清理函數已定義


## 階段三：清理公司資料 (COMPANY_INFO)

依 `company_name` 分組、彙總 `job_category`；產業依序 **公司名稱** → **job_category fallback** → **未分類**。`company_size` / `location` 皆 NULL，對齊 ERD 後去重。

In [15]:
# 依 company_name 分組，彙總 job_category（供產業 fallback）
def _agg_job_cats(x):
    parts = x.dropna().astype(str).str.strip()
    parts = parts[parts.str.len() > 0].unique()
    return " | ".join(parts) if len(parts) else ""

company_agg = df_raw.groupby("company_name", as_index=False).agg(
    company_name=("company_name", "first"),
    job_categories=("job_category", _agg_job_cats),
)
company_agg["company_name"] = company_agg["company_name"].apply(clean_text)
company_agg = company_agg[company_agg["company_name"].notna() & (company_agg["company_name"].str.len() > 0)]

# 產業：公司名稱 → job_category 彙總 fallback → 未分類
def _resolve_industry(row):
    ind = extract_industry(row["company_name"])
    if ind is not None:
        return ind
    ind = infer_industry_from_job_category(row["job_categories"])
    if ind is not None:
        return ind
    return "未分類"

company_agg["industry"] = company_agg.apply(_resolve_industry, axis=1)
company_agg["company_size"] = None
company_agg["location"] = None
company_agg["website"] = None
company_agg["description"] = None

df_company = company_agg[["company_name", "industry", "company_size", "location", "website", "description"]].drop_duplicates(subset=["company_name"]).reset_index(drop=True)

print(f"✓ 公司主檔 {len(df_company):,} 家 | 有 industry（非未分類）: {(df_company['industry'] != '未分類').sum():,}")
df_company.head(10)

# industry 各產業類別比例（供檢查）
industry_dist = df_company["industry"].value_counts().sort_index()
industry_check = pd.DataFrame({
    "industry": industry_dist.index,
    "家數": industry_dist.values,
    "比例%": (industry_dist.values / len(df_company) * 100).round(2),
})
print("\n【industry 產業類別比例】")
display(industry_check)

✓ 公司主檔 1,611 家 | 有 industry（非未分類）: 1,578

【industry 產業類別比例】


,industry,家數,比例%
0,商業,1,0.06
1,服務業,7,0.43
2,未分類,33,2.05
3,製造業,72,4.47
4,設計,1,0.06
5,資訊科技,1415,87.83
6,醫療,41,2.55
7,金融業,41,2.55


In [16]:
# 測試關鍵案例
test_cases = [
    "日月光半導體製造股份有限公司",  # 應該是「製造業」
    "中山醫學大學附設醫院",          # 應該是「醫療」
    "台積電科技股份有限公司",        # 應該是「製造業」（不是資訊科技）
    "趨勢科技股份有限公司",          # 應該是「資訊科技」
]

for company in test_cases:
    result = extract_industry(company)
    print(f"{company[:30]:30s} → {result}")

日月光半導體製造股份有限公司                 → 製造業
中山醫學大學附設醫院                     → 醫療
台積電科技股份有限公司                    → 製造業
趨勢科技股份有限公司                     → 資訊科技


## 階段四：清理職缺資料 (JOB_POSTING)

清理標題、描述、合併要求、標準化地點 (full_address, city, district)、薪資、遠端選項、job_details、日期。去重：有 `source_url` 用 (source_url, job_title)，否則 (company_name, job_title, full_address)。移除關鍵欄位空值。

In [6]:
# 複製並保留 company_name（稍後對應 company_id）
df_jobs = df_raw.copy()
df_jobs["company_name"] = df_jobs["company_name"].apply(clean_text)
df_jobs["job_title"] = df_jobs["job_name"].apply(clean_text)
df_jobs["job_description"] = df_jobs["job_description"].apply(clean_text)

# 合併要求
df_jobs["requirements"] = df_jobs.apply(merge_requirements, axis=1)

# 標準化地點：(full_address, city, district)。CSV 僅有 location
loc_out = df_jobs.apply(lambda r: standardize_location(None, None, r.get("location")), axis=1)
df_jobs["full_address"] = [x[0] for x in loc_out]
df_jobs["city"] = [x[1] for x in loc_out]
df_jobs["district"] = [x[2] for x in loc_out]

# 薪資
sal_out = df_jobs["salary"].apply(clean_salary)
df_jobs["salary_min"] = [x[0] for x in sal_out]
df_jobs["salary_max"] = [x[1] for x in sal_out]

# 遠端選項、job_details
df_jobs["remote_option"] = df_jobs.apply(lambda r: determine_remote_option(r.get("full_address"), r.get("job_type")), axis=1)
df_jobs["job_details"] = df_jobs.apply(create_job_details, axis=1)

# 日期：update_date -> posted_date, created_at -> scraped_at
df_jobs["posted_date"] = pd.to_datetime(df_jobs["update_date"], errors="coerce").dt.date
df_jobs["scraped_at"] = pd.to_datetime(df_jobs["created_at"], errors="coerce")

# 來源
df_jobs["source_platform"] = "104人力銀行"
df_jobs["source_url"] = df_jobs["url"].where(df_jobs["url"].notna() & (df_jobs["url"].astype(str).str.len() > 0))

# 固定欄位
df_jobs["is_active"] = True
df_jobs["is_embedded"] = False

print("✓ 職缺清理完成（未去重、未刪空）")
df_jobs[["company_name", "job_title", "full_address", "city", "district", "salary_min", "salary_max", "source_url", "job_description"]].head()

✓ 職缺清理完成（未去重、未刪空）


,company_name,job_title,full_address,city,district,salary_min,salary_max,source_url,job_description
0,泛太資訊科技開發股份有限公司,C#軟體工程師,台北市大安區光復南路102號7樓,台北市,大安區,40000.0,40000.0,https://www.104.com.tw/job/109g1?jobsource=job...,- 熟悉C#、.NET 、ASP.Net -依據系統需求規格進行程式設計與開發 -使用 C#...
1,凌群電腦股份有限公司,研發替代役-軟體工程師,台北市萬華區峨眉街115號6樓,台北市,萬華區,37000.0,37000.0,https://www.104.com.tw/job/1x89f?jobsource=job...,軟體開發與設計、架構設計、資料設計、系統整合、軟體產品研發、技術管理、專案管理 ※薪資依學經歷敘薪
2,瑞訊股份有限公司,系統工程師(高雄),高雄市大社區萬金路385巷3-2號,高雄市,大社區,35000.0,50000.0,https://www.104.com.tw/job/1xpg2?jobsource=job...,PLC 或 SCADA (Supervisory Control And Data Acqu...
3,商智資訊股份有限公司,資料庫程式工程師,台北市大安區復興南路2段363號3樓,台北市,大安區,35000.0,75000.0,https://www.104.com.tw/job/1yf7o?jobsource=job...,資料庫程式工程師 熟悉 SQL Server/Oracle/DB2 任一種資料庫操作
4,展達通訊股份有限公司_群光集團關係企業,軟體工程師(新竹),新竹市埔頂路27號3樓之1,新竹市,NaN,40000.0,40000.0,https://www.104.com.tw/job/21v0f?jobsource=job...,負責開發 IP-Cam 與其他 IoT 相關產品 之韌體或軟體，主要開發語言為 C / C+...


In [7]:
# 去重：有 source_url 用 (source_url, job_title)，否則 (company_name, job_title, full_address)
has_url = df_jobs["source_url"].notna() & (df_jobs["source_url"].astype(str).str.len() > 0)
df_with_url = df_jobs[has_url].drop_duplicates(subset=["source_url", "job_title"], keep="last")
df_no_url = df_jobs[~has_url].drop_duplicates(subset=["company_name", "job_title", "full_address"], keep="last")
df_jobs = pd.concat([df_with_url, df_no_url], ignore_index=True)

# 移除關鍵欄位為空
df_jobs = df_jobs[
    df_jobs["company_name"].notna() & (df_jobs["company_name"].astype(str).str.len() > 0) &
    df_jobs["job_title"].notna() & (df_jobs["job_title"].astype(str).str.len() > 0) &
    df_jobs["job_description"].notna() & (df_jobs["job_description"].astype(str).str.len() > 0)
].copy()

n_url_empty = (df_jobs["source_url"].isna() | (df_jobs["source_url"].astype(str).str.strip().str.len() == 0)).sum()
print(f"✓ 去重並移除空值後 {len(df_jobs):,} 筆職缺 | source_url 為空: {n_url_empty:,} 筆")

✓ 去重並移除空值後 2,596 筆職缺 | source_url 為空: 0 筆


## 階段五：資料驗證與統計

公司／職缺統計、**job_description** 與 **requirements** 字數分段（150以下｜150~300｜300~500｜500~800｜800~1200｜1200+，向量化建議 150~300）、必填欄位檢查。

In [8]:
# 公司
n_c = len(df_company)
n_c_ind = (df_company["industry"] != "未分類").sum()
print(f"【公司】 總數 {n_c:,} | 有明確產業（非未分類） {n_c_ind:,} | company_size/location 皆 NULL")

# 職缺
n_j = len(df_jobs)
print(f"【職缺】 總數 {n_j:,}")
print(f"  有 job_description: {df_jobs['job_description'].notna().sum():,} | 有 requirements: {df_jobs['requirements'].notna().sum():,}")
print(f"  有 salary_min: {df_jobs['salary_min'].notna().sum():,} | 有 full_address: {df_jobs['full_address'].notna().sum():,}")

# job_description 字數統計（向量化建議 150~300 字）
desc_len = df_jobs["job_description"].fillna("").str.len()
print(f"【job_description 字數】 平均 {desc_len.mean():.0f} 字")
print(f"  150以下: {(desc_len < 150).sum():,} | 150~300: {((desc_len >= 150) & (desc_len <= 300)).sum():,} | 300~500: {((desc_len > 300) & (desc_len <= 500)).sum():,} | 500~800: {((desc_len > 500) & (desc_len <= 800)).sum():,} | 800~1200: {((desc_len > 800) & (desc_len <= 1200)).sum():,} | 1200+: {(desc_len > 1200).sum():,}")

# requirements 字數統計（同向量化建議）
req_len = df_jobs["requirements"].fillna("").str.len()
print(f"【requirements 字數】 平均 {req_len.mean():.0f} 字")
print(f"  150以下: {(req_len < 150).sum():,} | 150~300: {((req_len >= 150) & (req_len <= 300)).sum():,} | 300~500: {((req_len > 300) & (req_len <= 500)).sum():,} | 500~800: {((req_len > 500) & (req_len <= 800)).sum():,} | 800~1200: {((req_len > 800) & (req_len <= 1200)).sum():,} | 1200+: {(req_len > 1200).sum():,}")

# 必填：company_name, job_title, job_description 已於階段四移除空值
missing = df_jobs[["company_name", "job_title", "job_description"]].isna().any(axis=1).sum()
print(f"  必填欄位缺失: {missing} 筆")

【公司】 總數 1,611 | 有明確產業（非未分類） 1,588 | company_size/location 皆 NULL
【職缺】 總數 2,596
  有 job_description: 2,596 | 有 requirements: 2,596
  有 salary_min: 2,596 | 有 full_address: 2,596
【job_description 字數】 平均 387 字
  150以下: 886 | 150~300: 636 | 300~500: 456 | 500~800: 318 | 800~1200: 161 | 1200+: 139
【requirements 字數】 平均 198 字
  150以下: 1,226 | 150~300: 922 | 300~500: 323 | 500~800: 95 | 800~1200: 21 | 1200+: 9
  必填欄位缺失: 0 筆


### 檢閱清理後 JD

以下顯示清理後的 `job_description`、`requirements` 完整內容（前 10 筆），可捲動檢視實際長相。

In [9]:
# 檢閱清理後 JD：完整顯示 job_description，可捲動檢視
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 20)
display(df_jobs[["company_name", "job_title", "job_description", "requirements"]].head(5))
pd.reset_option("display.max_colwidth")
pd.reset_option("display.max_rows")

,company_name,job_title,job_description,requirements
0,泛太資訊科技開發股份有限公司,C#軟體工程師,- 熟悉C#、.NET 、ASP.Net -依據系統需求規格進行程式設計與開發 -使用 C# 語言開發應用程式功能模組 -撰寫 ASP.NET Core Web API 服務 -開發 ASP.NET Core MVC 網頁應用系統 - 本職務可能之工作內容包括軟體產品研發、客製專案開發、現有系統維護,5年以上\n專科以上\n不拘\n中文 -- 聽 /精通、說 /精通、讀 /精通、寫 /精通\n英文 -- 聽 /略懂、說 /略懂、讀 /中等、寫 /略懂\n提升語文能力\n提升語文能力\n軟體程式設計、資料庫程式設計\nWindows 2003、Windows XP、ASP.NET、C#、Visual Basic .net、Visual C#、WinForm、ANSI SQL、MS SQL、ODBC、RDBMS、HTML、JavaScript
1,凌群電腦股份有限公司,研發替代役-軟體工程師,軟體開發與設計、架構設計、資料設計、系統整合、軟體產品研發、技術管理、專案管理 ※薪資依學經歷敘薪,不拘\n碩士\n資訊工程相關、資訊管理相關、數學及電算機科學學科類\n英文 -- 聽 /中等、說 /中等、讀 /中等、寫 /中等\n提升英文能力\n提升英文能力\n1.研究所以上資訊相關系所畢\n2.熟稔Java或.Net語言\n3.具系統分析設計與程式撰寫能力\n4.具團隊合作及溝通協調能力
2,瑞訊股份有限公司,系統工程師(高雄),"PLC 或 SCADA (Supervisory Control And Data Acquisition) 為具有""監控""和""資料擷取""功能的自動化系統 電力監控系統舉凡電子廠、石油、化工、汽車生產業都是其應用領域 我們主要設計、製作及維護 ＂PLC或電力監控系統 ＂專案 小型系統如水、電錶抄表系統，大型如全廠供電系統，變電所監控，發電廠電力監控都有實績 本職位主要工作內容： 1.使用圖控軟體設計、開發客製化的監控系統。 2.通訊設定,測試及修改。 3.配合專案地點，會有出差的機會。 未來本部門將以PLC及SCADA系統為核心，擴展成此類型專業團隊 非常歡迎具有此類工作有興趣(有相關經驗更佳)，加入我們的團隊！ 我們會依照您的工作經驗安排教育訓練計畫，讓您在這個職位工作勝任愉快!",不拘\n高中、專科、大學\n電機電子工程相關、資訊工程相關、資訊管理相關\n英文 -- 聽 /略懂、說 /略懂、讀 /略懂、寫 /略懂\n提升英文能力\n提升英文能力\n作業系統基本操作、資料備份與復原、資訊設備環境設定\nWindows Server 2019、C++、Visual Basic、MS SQL、PLC\n有其他專項-可面談
3,商智資訊股份有限公司,資料庫程式工程師,資料庫程式工程師 熟悉 SQL Server/Oracle/DB2 任一種資料庫操作,"1年以上\n大學以上\n不拘\n英文 -- 聽 /中等、說 /中等、讀 /中等、寫 /中等\n提升英文能力\n提升英文能力\nDB2、MS SQL、Oracle\n1. 熟資料庫SQL 語法, 指令及運用\n2. 有 Store Procedure coding 一年(含)以上經驗\n3. 熟悉 SQL Server/Oracle/DB2 任一種資料庫操作"
4,展達通訊股份有限公司_群光集團關係企業,軟體工程師(新竹),負責開發 IP-Cam 與其他 IoT 相關產品 之韌體或軟體，主要開發語言為 C / C++。 工作項目包含： 1.設備底層韌體開發 A. 提供生產測試所需的韌體功能 B. 支援 EMI / FCC 等相關認證測試所需功能 C. 提供客戶端應用程式介接的韌體支援 2.硬體驅動程式 (Device Driver) 的開發、移植與驗證 3.在 Linux / RTOS 平台上進行除錯與效能最佳化 4.處理並追蹤 客戶端問題回報，持續改善產品品質 5.主管交辦的其他相關事項 我們正在尋找對 嵌入式系統 (Embedded System) 開發具熱忱，並擁有實務經驗的夥伴。,1年以上\n大學、碩士\n資訊工程相關、電機電子工程相關、通信學類\n英文 -- 聽 /中等、說 /中等、讀 /中等、寫 /中等\n提升英文能力\n提升英文能力\n1.一年以上 網路產品或軟體開發經驗\n2.具 IP-Cam 韌體開發經驗 者尤佳\n3.熟悉 Internet 通訊協定 (TCP/IP、HTTP、RTSP 等) 者佳\n4.具 影像 / 聲音處理 相關產品開發經驗者佳\n5.具 電池供電類產品 之開發經驗者佳


## 階段六：準備 Supabase 連線

載入 `.env`（`Erd/.env` 或 `project_url` / `service_role_key`）、建立 Supabase 客戶端、測試連線。

In [10]:
from dotenv import load_dotenv
import os
from pathlib import Path

# 依序嘗試 .env 路徑
for p in [Path("Erd/.env"), Path(".env"), Path("supabase_control/Erd/.env")]:
    if p.exists():
        load_dotenv(p)
        print(f"✓ 載入 {p}")
        break
else:
    load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL") or os.getenv("project_url")
SUPABASE_KEY = os.getenv("SUPABASE_KEY") or os.getenv("SUPABASE_SERVICE_ROLE_KEY") or os.getenv("service_role_key")

if not SUPABASE_URL or not SUPABASE_KEY:
    raise ValueError("請在 .env 設定 project_url + service_role_key 或 SUPABASE_URL + SUPABASE_KEY")

from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

# 測試連線（依實際表名調整，例如 company_info）
_ = supabase.table("company_info").select("company_id").limit(1).execute()
print("✓ Supabase 連線成功")

✓ 載入 Erd\.env
✓ Supabase 連線成功


## 階段七：寫入 Supabase 資料庫

依 ERD 嚴格寫入 `company_info` → `job_posting`。`job_posting` 僅有 **full_address, city, district**（無 `location`），寫入三者。寫入職缺前依 **source_url + job_title**（有 URL）或 **company_id + job_title + full_address**（無 URL）檢查重複，重複則跳過。

In [ ]:
import pandas as pd

# 1. 向量化清洗
def clean_dataframe(df):
    # 使用新版 map 取代 applymap
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.where(pd.notnull(df), None).replace("", None)
    return df

df_company = clean_dataframe(df_company)

# 2. 獲取現有資料
existing_data = supabase.table("company_info").select("company_name", "company_id").execute()
company_name_to_id = {d["company_name"]: d["company_id"] for d in existing_data.data}

# 3. 過濾新資料
new_mask = ~df_company["company_name"].isin(company_name_to_id.keys())
df_new_companies = df_company[new_mask].drop_duplicates(subset=["company_name"])

# 4. 批量插入與精確統計
new_count = len(df_new_companies)
success_count = 0
fail_count = 0

if new_count > 0:
    try:
        to_insert = df_new_companies.to_dict(orient="records")
        ins_res = supabase.table("company_info").insert(to_insert).execute()
        
        if ins_res.data:
            success_count = len(ins_res.data)
            fail_count = new_count - success_count  # 預計減實際 = 失敗
            for d in ins_res.data:
                company_name_to_id[d["company_name"]] = d["company_id"]
    except Exception as e:
        fail_count = new_count
        print(f"❌ 批量寫入發生嚴重錯誤: {e}")

# 5. 最終結果彙報
db_total = len(company_name_to_id)
print(f"--- 處理回報 ---")
print(f"✓ 成功新增: {success_count} 筆")
print(f"⚠️ 寫入失敗: {fail_count} 筆")
print(f"📊 資料庫目前總計: {db_total} 家公司")

--- 處理回報 ---
✓ 成功新增: 611 筆
⚠️ 寫入失敗: 0 筆
📊 資料庫目前總計: 1611 家公司


In [18]:
import math
import pandas as pd

# 1. 向量化清洗與預處理 (對齊 ERD 資料型態)
def clean_jobs(df):
    # 使用 map 處理字串，並統一空值
    df = df.map(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.where(pd.notnull(df), None)
    
    # 確保薪資為 INT (對齊 ERD: salary_min/max 為 INT)
    df["salary_min"] = pd.to_numeric(df["salary_min"], errors='coerce').fillna(0).astype(int)
    df["salary_max"] = pd.to_numeric(df["salary_max"], errors='coerce').fillna(0).astype(int)
    
    # 處理布林值 (對齊 ERD: is_active 為 BOOLEAN)
    df["is_active"] = df["is_active"].map(lambda x: True if x is None else bool(x))
    return df

df_jobs = clean_jobs(df_jobs)

# 2. 批量獲取 ID 與指紋 (解決 KeyError 的關鍵)
# 確保 select 的欄位名稱與 ERD 完全一致
existing_data = supabase.table("job_posting").select("source_url", "job_title", "company_id", "full_address").execute()
db_rows = existing_data.data if existing_data.data else []

# 使用 .get() 安全存取，避免 KeyError [修正點]
existing_fingerprints_url = {
    (d.get("source_url"), d.get("job_title")) 
    for d in db_rows 
    if d.get("source_url") and d.get("job_title")
}

existing_fingerprints_addr = {
    (d.get("company_id"), d.get("job_title"), d.get("full_address")) 
    for d in db_rows 
    if not d.get("source_url") and d.get("job_title")
}

# 3. 準備批量 Payload（date/datetime→ISO；nan→None，否則 JSON 序列化失敗）
def _serialize_dt(v):
    if v is None or (hasattr(v, "__float__") and pd.isna(v)):
        return None
    if hasattr(v, "isoformat"):
        return v.isoformat()
    return str(v) if v is not None else None

def _json_safe(v):
    """將 nan/NaT 轉 None、date/datetime 轉 ISO，確保 JSON 可序列化。"""
    if v is None:
        return None
    if isinstance(v, float) and (math.isnan(v) or math.isinf(v)):
        return None
    if hasattr(v, "__float__") and pd.isna(v):
        return None
    if hasattr(v, "isoformat"):
        return v.isoformat()
    if isinstance(v, dict):
        return {k: _json_safe(vv) for k, vv in v.items()}
    return v

to_insert = []
job_skip = 0

for _, row in df_jobs.iterrows():
    cname = row.get("company_name")
    cid = company_name_to_id.get(cname)
    
    # 若找不到公司 ID，視為失敗或跳過 (對齊 ERD 關聯約束)
    if not cid:
        continue
        
    src_url = row.get("source_url")
    title = row.get("job_title")
    addr = row.get("full_address")

    # 本地重複判定
    if src_url:
        if (src_url, title) in existing_fingerprints_url:
            job_skip += 1
            continue
    else:
        if (cid, title, addr) in existing_fingerprints_addr:
            job_skip += 1
            continue

    # 封裝資料 (嚴格對齊 5.2 JOB_POSTING 欄位)
    payload = {
        "company_id": cid,
        "job_title": title,
        "job_description": row.get("job_description"),
        "requirements": row.get("requirements"),
        "salary_min": int(row["salary_min"]),
        "salary_max": int(row["salary_max"]),
        "full_address": addr,
        "city": row.get("city"), # ERD 5.2
        "district": row.get("district"), # ERD 5.2
        "remote_option": row.get("remote_option"),
        "job_details": row.get("job_details") if isinstance(row.get("job_details"), dict) else None,
        "source_platform": row.get("source_platform") or "104人力銀行",
        "source_url": src_url,
        "posted_date": _serialize_dt(row.get("posted_date")),
        "scraped_at": _serialize_dt(row.get("scraped_at")),
        "is_active": row.get("is_active"),
    }
    payload = {k: _json_safe(v) for k, v in payload.items()}
    to_insert.append(payload)

# 4. 批量寫入與統計
job_ok = 0
job_err = 0
if to_insert:
    try:
        # 500 筆一抽，效率與穩定的平衡點
        for i in range(0, len(to_insert), 500):
            batch_data = to_insert[i:i+500]
            ins_res = supabase.table("job_posting").insert(batch_data).execute()
            if ins_res.data:
                job_ok += len(ins_res.data)
    except Exception as e:
        print(f"❌ 批量寫入錯誤: {e}")
        job_err = len(to_insert) - job_ok

# 5. 最終結果彙報
print(f"--- 職缺處理回報 ---")
print(f"✓ 成功寫入: {job_ok} 筆")
print(f"⏭️ 跳過重複: {job_skip} 筆")
print(f"❌ 寫入失敗: {job_err} 筆")
print(f"📊 資料庫目前職缺總數: {len(db_rows) + job_ok} 筆")

--- 職缺處理回報 ---
✓ 成功寫入: 2596 筆
⏭️ 跳過重複: 0 筆
❌ 寫入失敗: 0 筆
📊 資料庫目前職缺總數: 2596 筆


## 階段八：匯出清理後資料（備份）

匯出 `companies_cleaned.csv`、`jobs_cleaned.csv`，編碼 `utf-8-sig`。

In [19]:
out_company = DATA_DIR / "companies_cleaned.csv"
out_jobs = DATA_DIR / "jobs_cleaned.csv"

df_company.to_csv(out_company, index=False, encoding="utf-8-sig")
df_jobs.to_csv(out_jobs, index=False, encoding="utf-8-sig")

print(f"✓ 已匯出 {out_company} ({len(df_company):,} 筆)")
print(f"✓ 已匯出 {out_jobs} ({len(df_jobs):,} 筆)")

✓ 已匯出 companies_cleaned.csv (1,611 筆)
✓ 已匯出 jobs_cleaned.csv (2,596 筆)
